<a href="https://colab.research.google.com/github/Jaisman/cloth_classifier/blob/main/geekhaven_contest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'geek-haven-ml-contest:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F84613%2F9497287%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241009%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241009T142331Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5ee05049bad417004084b79f0c4ea3af78013cdbfaa0728d2528f531e0635d2f9e700dc32f64fb5a95efe7f9304d9a7d0e0585e89cf3241d1f01cb38eb90b53c55c12d57908b4c36a6202834c4c38947435b3f640bf8e2f256f6852d74e84b5716e9f949e7718d7cb3673cf92dfaa67c892c89dc1213b0d2de945ec7f3e31538b0767fa8b8293f434db2475eac781c644f8c767126ee085742f52508d7fefb83a3ccc23d04a47142f623c97a74cee027cfb3ef8a4648644519492af39489a946cff1702760211c8d0061532173a6feb9e85ab3affc023dfc919d77a6453440a1d70880c7fcfdf2a98aee498e82fb9bc5fabc0352ceba65aa82589e67a56471a1,testcsv:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5638823%2F9310806%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241009%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241009T142331Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1598a2843058a984b4139d51b012eccf37fb3f5429a170594e13375108af000dd41573a2fdac3624947e6cb7c96692e12f34a408373d8103ddab56cd3a61aa564ddd0d6065e38b18869e4a20179fbdf77134f84cac33b6f2ca091511522eba020b4f8afe4ca1f4ecf53cad85113f9fe84df161c870c7af386889c462d6a3539b2dd2bfe20d89fa2ca31b5c2d1887c837094499121ae62efa280821263a455bb906df5e3f5c31e6c9863b7d61b6d24f73627f4f4ed39b72d0e7e895b3f993c2f8fe3667f35c243311846e2906e96f34c10a9d7db82c188a76efcfc8384ae3d44f74c2e8621b8894319800cd9da25b06e1273033bfbecfbe7e5b3408c45f36d52e,traincsv:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5638863%2F9310859%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241009%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241009T142331Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D82808cfde67dab6bee22292ca2113b814b7fbdf0f08bc4d588a8f620310f35b4da2108e39931679e57f2aa08f257e9fc1c4f8fa920b3daf64afe9b9ed013ef24b6d2b4fb92a73a38b550779283bfbb457c24981d29276369cfbda5e413339f4ec79736988cacdb75cb48adbafd33a71fc0d5d7889e76f756934203464fae1be925d0813e5c93f67c91613f21b183b9c9719809f79a424305a6265b03ec44cb481d97173899ff596aedcb1bf5e25858911e53fdfbb5f6377abb321985d748741d272a39952adaf28345f35bbd353ff454e257f1b2d3db47af3676da785ad9ba2bea6df8c88509edf3fb5f3eacb06d665d7c8ebead05b1c1e3868ff9b9195443a8'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 548668 bytes downloaded
Downloaded and uncompressed: geek-haven-ml-contest
[==================================================] 108400 bytes downloaded
Downloaded and uncompressed: testcsv
[==================================================] 440290 bytes downloaded
Downloaded and uncompressed: traincsv
Data source import complete.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

In [ ]:
data = pd.read_csv('/kaggle/input/geek-haven-ml-contest/train_df.csv')

In [ ]:
data['CombinedText'] = data['ProductName'] + ' ' + data['Description']
X = data['CombinedText']
y = data['Category']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
vectoriser = TfidfVectorizer(max_features=5000)
X_train_ = vectoriser.fit_transform(X_train)
X_test_ = vectoriser.transform(X_test)


In [ ]:
classifier = RandomForestClassifier(n_estimators=500, random_state=42)

In [ ]:
classifier.fit(X_train_,y_train)

RandomForestClassifier(n_estimators=500, random_state=42)

In [ ]:
y_pred = classifier.predict(X_test_)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
print(accuracy)

0.9508816120906801


In [ ]:
data_test = pd.read_csv('/kaggle/input/geek-haven-ml-contest/test.csv')

In [ ]:
data_test['Combined'] = data_test['ProductName']+' '+ data_test['Description']

In [ ]:
X_test_d = data_test['Combined']

In [ ]:
X_test_d = vectoriser.transform(X_test_d)

In [ ]:
y_test_d = classifier.predict(X_test_d)

In [ ]:
import pandas as pd
output_df = pd.DataFrame({
    'ProductName': data_test['ProductName'],
    'Description': data_test['Description'],
    'Category': y_test_d
})

output_df.to_csv('submission.csv', index=False)
